In [25]:
import numpy as np
import pandas as pd
import pymongo
import bson
from tqdm import tqdm

In [26]:
class MongoServer():
    credentials = None
    conn = None
    db = None
    collections = {}
    def __init__(self, run = False, credentials = "credentials.txt", db_name = "agile_data_science_group_3"):
        if run:
            if not self.connect2Mongo(credentials):
                print("Connection to server Failed.")
            
            if not self.connect2DataBase(db_name):
                print("Connection Data Base Failed.")
    """Rotine to connect to Mongo DB"""    
    def connect2Mongo(self, credentials = "credentials.txt"):
        try:
            #use your database name, user and password here:
            with open(credentials, 'r', encoding='utf-8') as f:
                [name,password,url,dbname]=f.read().splitlines()
            conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
            self.conn = conn
            return True
        except pymongo.errors.ConnectionFailure as e:
            return False
    
    """Routine to connect to a Data Base"""
    def connect2DataBase(self, db_name = "agile_data_science_group_3"):
        try:
            self.db = self.conn[db_name]
            return True
        except:
            return False
    
    """Return the available collections in a data base"""
    def listOfCollections (self):
        return self.db.collection_names()
    
    """Donwload all the ollections from the data base"""
    def getAllCollections (self):
        collections = self.listOfCollections()
        for col in collections:
            self.getCollectionFromServer(col)
        return True
    
    
    """Routine to get one collection of the Data Base"""
    # return the collection
    def getCollectionFromServer (self, name_collection):
        if name_collection in self.db.collection_names():
            self.collections[name_collection] = self.db.get_collection(name_collection)
            print ("Collection ",name_collection," Update in Local.")
            return True
        return False
    
    """ Take the collection from the local copy"""
    def getCollection(self, name_collection):
        if not name_collection in self.collections:
            if not self.getCollectionFromServer(name_collection):
                return False
        return self.collections[name_collection]
    
    """ Gets all the items of the collection"""
    def getItems(self, name_collection):
        if not name_collection in self.collections:
            if not self.getCollectionFromServer(name_collection):
                return False
        return [element for element in self.collections[name_collection].find()]
    
    """Query in one Collection"""
    def searchInCollection(self, name_collection, field, patro):
        return [element for element in self.collections[name_collection].find({field:patro})]
    
    """Query in all the Collections"""
    def searchInDB (self, field, patro):
        query = {}
        for collection in self.db.collection_names():
            query[collection] = [element for element in self.db.get_collection(collection).find({field:patro})]
        return query
    
    """Find one in the collection"""
    def findOne(self, collection_name):
        return self.db.get_collection(collection_name).find_one()
    
    """Froce Brute search"""
    def searchInDBForceBrute(self, patro):
        query = {}
        for collection in self.db.collection_names():
            query[collection] = []
            
            self.searchInCollection(collection, )
            print(self.getItems(collection))
            for element in self.getItems(collection)[0]:
                for key in element.keys():
                    query[collection] = [element for element in self.db.get_collection(collection).find({field:patro})]
                    
    def findNElement(self, collection_name, N):
        query = []
        for item in self.db.get_collection(collection_name).find().limit( N ):
            query.append(item)
        return query
    
    def insertInCollection (self, collection_name,  item):
        assert type(item) == dict, "Item must be a dictionary"
        return self.db.get_collection(collection_name).insert_one(item)
        
                
        
MS = MongoServer(True)   


In [52]:
MS.searchInCollection('recipes', 'id', name)[0]['_id']

ObjectId('59fb0d77eee3642f3c0684bd')

In [53]:
for item in tqdm(MS.getItems('recipes_ingredients')):
    ingredients = list(item.values())[1]
    name = list(item)[1]
    recepie_id = MS.searchInCollection('recipes', 'id', name)[0]['_id']
    
    MS.insertInCollection('RecIng', {'recipe_id': recepie_id, 'ingredients':ingredients})

100%|██████████████████████████████████████████████████████████████████████████████| 2467/2467 [05:22<00:00,  7.65it/s]


In [63]:
MS.searchInCollection('RecIng', '_id', bson.objectid.ObjectId('5a306ed890ea86097ce0dbe6'))

[{'_id': ObjectId('5a306ed890ea86097ce0dbe6'),
  'ingredients': ['ghee',
   'oil',
   'onion',
   'pack',
   'asda',
   'chicken',
   'thigh',
   'fillet',
   'skin',
   'garlic',
   'clove',
   'ginger',
   'turmeric',
   'cumin',
   'asda',
   'garam',
   'masala',
   'chillie',
   'can',
   'coconut',
   'milk',
   'pumpkin',
   'skin',
   'fibre',
   'cornflour',
   'cold',
   'water',
   'make',
   'paste',
   'coriander',
   'rice',
   'serve'],
  'recipe_id': ObjectId('59fb0d77eee3642f3c0685cd')}]

In [60]:
{"$and": [{"borough": "Manhattan"}, {"grades": {"$elemMatch": {"grade": "A"}}}]}

{'ingredients': 'cold'}

In [61]:
for i in MS.getCollection('RecIng').find({'ingredients':'can'},{ 'ingredients':'cold'}):
    print (i)
    print("-------------------")

{'_id': ObjectId('5a306ed890ea86097ce0dbe6'), 'ingredients': ['ghee', 'oil', 'onion', 'pack', 'asda', 'chicken', 'thigh', 'fillet', 'skin', 'garlic', 'clove', 'ginger', 'turmeric', 'cumin', 'asda', 'garam', 'masala', 'chillie', 'can', 'coconut', 'milk', 'pumpkin', 'skin', 'fibre', 'cornflour', 'cold', 'water', 'make', 'paste', 'coriander', 'rice', 'serve']}
-------------------
{'_id': ObjectId('5a306ed990ea86097ce0dbe9'), 'ingredients': ['pack', 'smash', 'instant', 'mash', 'potato', 'can', 'asda', 'tuna', 'chunk', 'water', 'chive', 'pinch', 'cayenne', 'egg', 'beaten', 'butter', 'asda', 'cornflake', 'breadcrumb', 'filippo', 'berio', 'spray', 'oil', 'asparagu', 'serve', 'pea', 'serve', 'sweetcorn', 'serve']}
-------------------
{'_id': ObjectId('5a306ed990ea86097ce0dbea'), 'ingredients': ['banana', 'frozen', 'solid', 'can', 'ambrosia', 'light', 'devon', 'custard', 'frozen', 'hr', 'pot', 'yeo', 'valley', 'fat', 'greek', 'style', 'honey', 'asda', 'natural', 'food', 'green', 'frozen', 'rasp

{'_id': ObjectId('5a306ee790ea86097ce0dc5d'), 'ingredients': ['asda', 'meringue', 'nest', 'broken', 'piece', 'can', 'asda', 'pear', 'halve', 'syrup', 'pot', 'asda', 'greek', 'style', 'yogurt', 'drop', 'asda', 'special', 'vanilla', 'extract', 'honey', 'dark', 'chocolate']}
-------------------
{'_id': ObjectId('5a306ee890ea86097ce0dc6a'), 'ingredients': ['onion', 'sunflower', 'oil', 'asda', 'tikka', 'curry', 'paste', 'can', 'coconut', 'milk', 'pack', 'asda', 'chicken', 'thigh', 'fillet', 'piece', 'slit', 'lentil', 'sweet', 'potato', 'bag', 'leaf', 'spinach', 'naan', 'bread', 'rice', 'serve']}
-------------------
{'_id': ObjectId('5a306ee890ea86097ce0dc6b'), 'ingredients': ['sunflower', 'oil', 'onion', 'clove', 'garlic', 'hot', 'vegetable', 'stock', 'made', 'cube', 'carrot', 'long', 'strip', 'resemble', 'noodle', 'special', 'exotic', 'mushroom', 'pack', 'cauldron', 'original', 'tofu', 'can', 'bamboo', 'shoot', 'light', 'soy', 'sauce', 'wine', 'vinegar', 'lime', 'juice', 'lime', 'zest', 'c

{'_id': ObjectId('5a306efc90ea86097ce0dd0e'), 'ingredients': ['pack', 'lamb', 'leg', 'steak', 'wine', 'beef', 'stock', 'onion', 'clove', 'garlic', 'herb', 'can', 'tomatoe', 'tomato', 'purée', 'pasta', 'cornflour', 'water', 'cherry', 'tomatoe', 'frozen', 'pea', 'carrot', 'courgette', 'thin', 'strip', 'fat', 'mascarpone', 'parsley']}
-------------------
{'_id': ObjectId('5a306efd90ea86097ce0dd13'), 'ingredients': ['onion', 'olive', 'oil', 'garlic', 'clove', 'thyme', 'flat-leaf', 'parsley', 'pinch', 'sugar', 'asda', 'shortcrust', 'pastry', 'flour', 'can', 'asda', 'anchovie', 'asda', 'special', 'kalamata', 'olive']}
-------------------
{'_id': ObjectId('5a306efe90ea86097ce0dd1c'), 'ingredients': ['carrot', 'pepper', 'onion', 'oil', 'piece', 'celery', 'can', 'napolina', 'tomatoe', 'tomato', 'purée', 'oregano', 'slice', 'bread', 'day', 'old', 'crust', 'milk', 'lean', 'turkey', 'mince', 'spaghetti']}
-------------------
{'_id': ObjectId('5a306eff90ea86097ce0dd23'), 'ingredients': ['can', 'ch

{'_id': ObjectId('5a306f1390ea86097ce0ddc7'), 'ingredients': ['sunflower', 'oil', 'chosen', 'cajun', 'pack', 'butcher’', 'prime', 'beef', 'rump', 'steak', 'can', 'chosen', 'bean', 'tomato', 'sauce', 'lea', 'perrin', 'worcestershire', 'sauce', 'dash', 'tabasco', 'pepper', 'sauce', 'optional', 'pack', 'asda', 'spinach']}
-------------------
{'_id': ObjectId('5a306f1490ea86097ce0ddcf'), 'ingredients': ['oil', 'frozen', 'scratch', 'cook', 'onion', 'frozen', 'scratch', 'cook', 'garlic', 'frozen', 'scratch', 'cook', 'italian', 'herb', 'mix', 'asda', 'frozen', 'vegetable', 'frozen', 'scratch', 'cook', 'casserole', 'root', 'veg', 'mix', 'can', 'tomatoe', 'reduced-salt', 'vegetable', 'stock', 'cube', 'spaghetti', 'broken', 'chunk', 'baguette', 'torn', 'chunk', 'cheddar']}
-------------------
{'_id': ObjectId('5a306f1590ea86097ce0ddd4'), 'ingredients': ['asda', 'chosen', 'fusilli', 'pasta', 'olive', 'oil', 'asda', 'smart', 'chicken', 'breast', 'fillet', 'piece', 'onion', 'pack', 'asda', 'les', '

{'_id': ObjectId('5a306f2790ea86097ce0de57'), 'ingredients': ['onion', 'sunflower', 'oil', 'pack', 'asda', 'lean', 'beef', 'steak', 'mince', 'chilli', 'powder', 'flour', 'can', 'asda', 'tomatoe', 'chilli', 'tomato', 'purée', 'sugar', 'can', 'kidney', 'bean', 'flour', 'asda', 'chosen', 'ready', 'roll', 'shortcrust', 'pastry', 'egg', 'beaten']}
-------------------
{'_id': ObjectId('5a306f2890ea86097ce0de5e'), 'ingredients': ['sheet', 'fine-leaf', 'gelatine', 'can', 'coconut', 'milk', 'milk', 'caster', 'sugar', 'lime', 'zest', 'asda', 'taste', 'pineapple', 'chunk', 'demerara', 'sugar']}
-------------------
{'_id': ObjectId('5a306f2a90ea86097ce0de70'), 'ingredients': ['sunflower', 'oil', 'onion', 'pack', 'butcher’', 'lean', 'beef', 'mince', 'mild', 'chilli', 'powder', 'carton', 'chosen', 'tomatoe', 'chillie', 'pepper', 'can', 'chosen', 'chilli', 'bean', 'dark', 'chocolate', 'cocoa', 'solid', 'lettuce', 'pack', 'old', 'el', 'paso', 'taco', 'shell', 'asda', 'fat', 'cream', 'serve']}
--------

...

{'_id': ObjectId('5a306faf90ea86097ce0e29b'), 'ingredients': ['fusilli', 'pasta', 'penne', 'pasta', 'onion', 'olive', 'oil', 'garlic', 'clove', 'pepper', 'can', 'tomatoe', 'chilli', 'flake', 'tomato', 'purée', 'asda', 'sweet', 'chilli', 'sauce', 'can', 'anchovie', 'dry', 'kitchen', 'paper', 'olive', 'crespo', 'caper', 'basil', 'prawn', 'asda', 'cracker', 'bite']}
-------------------
{'_id': ObjectId('5a306fb190ea86097ce0e2aa'), 'ingredients': ['onion', 'vegetable', 'oil', 'celeriac', 'carrot', 'hot', 'chicken', 'stock', 'can', 'asda', 'bean', 'tomato', 'sauce', 'asda', 'mexican', 'style', 'chicken', 'asda', 'sweet', 'chilli', 'sauce', 'asda', 'dark', 'soy', 'sauce', 'parsley', 'serve']}
-------------------
{'_id': ObjectId('5a306fb290ea86097ce0e2b0'), 'ingredients': ['olive', 'oil', 'asda', 'smart', 'pork', 'slice', 'skin', 'thick', 'strip', 'pack', 'asda', 'smart', 'sausage', 'bag', 'asda', 'smart', 'root', 'vegetable', 'pack', 'head', 'garlic', 'broken', 'clove', 'cox’', 'apple', 'sk

{'_id': ObjectId('5a306fc590ea86097ce0e355'), 'ingredients': ['pack', 'chosen', 'coconut', 'cookie', 'asda', 'digestive', 'biscuit', 'butter', 'plu', 'instant', 'coffee', 'optional', 'can', 'carnation', 'cook', 'caramel', 'banana', 'ripe', 'firm', 'lemon', 'juice', 'double', 'cream']}
-------------------
{'_id': ObjectId('5a306fc690ea86097ce0e358'), 'ingredients': ['butter', 'plu', 'light', 'soft', 'brown', 'sugar', 'ball', 'jar', 'stem', 'ginger', 'syrup', 'can', 'pear', 'juice', 'asda', 'flour', 'powder', 'ginger', 'cinnamon', 'free-range', 'egg']}
-------------------
{'_id': ObjectId('5a306fc690ea86097ce0e35d'), 'ingredients': ['pack', 'asda', 'special', 'pork', 'sausage', 'olive', 'oil', 'onion', 'carrot', 'garlic', 'clove', 'can', 'asda', 'tomatoe', 'herb', 'can', 'asda', 'tomatoe', 'chilli', 'pepper', 'sugar', 'spaghetti', 'parmesan', 'serve']}
-------------------
{'_id': ObjectId('5a306fc790ea86097ce0e364'), 'ingredients': ['can', 'reduced-fat', 'coconut', 'milk', 'chicken', 'br

{'_id': ObjectId('5a306fd990ea86097ce0e3fc'), 'ingredients': ['pack', 'asda', 'chicken', 'thigh', 'fillet', 'jar', 'levi', 'root', 'reggae', 'reggae', 'jerk', 'sauce', 'long-grain', 'rice', 'onion', 'sunflower', 'oil', 'solid', 'coconut', 'cream', 'can', 'asda', 'pinto', 'bean', 'thyme']}
-------------------
{'_id': ObjectId('5a306fda90ea86097ce0e401'), 'ingredients': ['onion', 'sweet', 'pepper', 'olive', 'oil', 'garlic', 'clove', 'honey', 'oregano', 'herb', 'can', 'asda', 'tomatoe', 'can', 'cannellini', 'bean', 'olive', 'fusilli', 'pasta', 'cod', 'portion']}
-------------------
{'_id': ObjectId('5a306fda90ea86097ce0e404'), 'ingredients': ['pack', 'asda', 'butcher', 'casserole', 'beef', 'pinche', 'dry', 'mustard', 'cornflour', 'onion', 'sprig', 'thyme', 'knorr', 'salt', 'beef', 'stock', 'cube', 'tomato', 'purée', 'chestnut', 'mushroom', 'sheet', 'filo', 'pastry', 'rest', 'pack', 'can', 'frozen', 'mild', 'olive']}
-------------------
{'_id': ObjectId('5a306fda90ea86097ce0e406'), 'ingred

{'_id': ObjectId('5a306fee90ea86097ce0e490'), 'ingredients': ['pack', 'asda', 'sweet', 'potato', 'pepper', 'piece', 'sunflower', 'oil', 'pack', 'asda', 'shitake', 'mushroom', 'can', 'asda', 'chick', 'pea', 'cumin', 'bag', 'leaf', 'spinach', 'sunflower', 'pack', 'merchant', 'gourmet', 'quinoa', 'grain', 'rice', 'mix', 'asda', 'good', 'chilli', 'coriander']}
-------------------
{'_id': ObjectId('5a306fee90ea86097ce0e492'), 'ingredients': ['olive', 'oil', 'onion', 'carrot', 'stick', 'celery', 'button', 'mushroom', 'clove', 'garlic', 'can', 'tomatoe', 'lentil', 'tomatoe', 'tomato', 'ketchup', 'oregano', 'vegetable', 'stock', 'spaghetti', 'basil', 'vegetarian', 'cheese', 'serve']}
-------------------
{'_id': ObjectId('5a306fef90ea86097ce0e49d'), 'ingredients': ['olive', 'oil', 'onion', 'butternut', 'squash', 'chantenay', 'carrot', 'top', 'clove', 'garlic', 'ginger', 'cumin', 'turmeric', 'cinnamon', 'tomato', 'purée', 'carton', 'asda', 'tomatoe', 'apricot', 'can', 'chickpea', 'vegetable', 's

{'_id': ObjectId('5a30700b90ea86097ce0e550'), 'ingredients': ['special', 'spanish', 'chorizo', 'casero', 'can', 'chosen', 'chickpea', 'water', 'onion', 'cherry', 'tomatoe', 'chosen', 'olive', 'onion']}
-------------------
{'_id': ObjectId('5a30700b90ea86097ce0e554'), 'ingredients': ['olive', 'oil', 'asda', 'chorizo', 'sausage', 'slice', 'pepper', 'carrot', 'clove', 'garlic', 'onion', 'kale', 'can', 'green', 'lentil', 'parsley', 'wine', 'vinegar']}
-------------------
{'_id': ObjectId('5a30700c90ea86097ce0e55b'), 'ingredients': ['pepper', 'colour', 'sachet', 'chosen', 'mediterranean', 'style', 'giant', 'cou', 'cou', 'vegetable', 'stock', 'made', 'stock', 'cube', 'sultana', 'tomatoe', 'pine', 'nut', 'can', 'haricot', 'bean', 'cannellini', 'bean', 'feta', 'cheese', 'salad', 'serve']}
-------------------
{'_id': ObjectId('5a30700d90ea86097ce0e560'), 'ingredients': ['courgette', 'matchstick', 'sweetcorn', 'can', 'well', 'egg', 'flour', 'feta', 'handful', 'rocket', 'olive', 'oil']}
---------